In [21]:
from keras.applications import vgg16
model = vgg16.VGG16(weights=None, input_shape=[201, 201, 2])

/usr/lib64/python3.6/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


In [23]:
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
    
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — val_f1: %f — val_precision: %f — val_recall %f" % (_val_f1, _val_precision, _val_recall))
        return
 
metrics = Metrics()

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
from dataGenerator import DataGenerator
dataGenerator = DataGenerator('../training', batch_size=128)

In [26]:
from keras import callbacks
tensorboard = callbacks.TensorBoard(log_dir='tensorboard')

In [27]:
model.fit_generator(dataGenerator, epochs=1)

Epoch 1/1


NotImplementedError: 